In [ ]:
import json
import time

import yfinance as yf
from kafka import KafkaProducer

In [ ]:
producer = KafkaProducer(
    bootstrap_servers='10.204.131.11:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

TICKERS = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'V']

In [ ]:
# fetch real-time data for all tickers
def fetch_data():
    data = {}
    for ticker in TICKERS:
        ticker_data = yf.download(ticker, interval="1m", period="1d")
        # Convert the last row of the DataFrame to a dictionary
        last_row = ticker_data.tail(1).to_dict(orient='records')[0]
        data[ticker] = last_row
    return data

In [ ]:
try:
    while True:
        data = fetch_data()

        for ticker, ticker_data in data.items():
            # Convert data to JSON and send it
            data_json = json.dumps({ticker: ticker_data})
            producer.send('g05in', data_json)
        # Wait 1 minute before fetching new data
        time.sleep(60)
        
except KeyboardInterrupt:
    producer.flush()
    producer.close()